In [1]:
import pandas as pd
import gzip
import json
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
meta_df = getDF('meta_Prime_Pantry.json.gz')
rows,columns=meta_df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)
meta_df.head()


('Number of rows: ', 10815)
('Number of columns: ', 11)


,asin,description,title,also_buy,image,price,details,brand,also_view,rank,feature
0,B00005BPJO,[Sink your sweet tooth into MILK DUDS Candya d...,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",[https://images-na.ssl-images-amazon.com/image...,$5.00,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",Milk Duds,NaN,NaN,NaN
1,B00005BPJO,[Sink your sweet tooth into MILK DUDS Candya d...,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",[https://images-na.ssl-images-amazon.com/image...,$5.00,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",Milk Duds,NaN,NaN,NaN
2,B0000DIF38,[A perfect Lentil soup starts with Goya Lentil...,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",[https://images-na.ssl-images-amazon.com/image...,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",Goya,"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",NaN,NaN
3,B0000DIWNI,[Saran Premium Wrap is an extra tough yet easy...,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",[https://images-na.ssl-images-amazon.com/image...,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",Saran,"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",NaN,NaN
4,B0000DIWNZ,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",NaN,[https://images-na.ssl-images-amazon.com/image...,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",Saran,[B0014CZ0TE],NaN,NaN


In [5]:
#Number of unique user id  in the data
print('Number of unique product in Raw data = ', meta_df['asin'].nunique())

('Number of unique product in Raw data = ', 10814)


In [6]:
#Dropping unused variables
meta_df.drop(['description', 'brand', 'rank'
, 'feature', 'image', 'details'],axis=1,inplace=True)

In [7]:
meta_df.head(10)

,asin,title,also_buy,price,also_view
0,B00005BPJO,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",$5.00,NaN
1,B00005BPJO,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",$5.00,NaN
2,B0000DIF38,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",NaN,"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81..."
3,B0000DIWNI,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",NaN,"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9..."
4,B0000DIWNZ,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",NaN,NaN,[B0014CZ0TE]
5,B0000GH6UG,"Ibarra Mexican Chocolate, 19 oz",[B008MHKWLK],NaN,NaN
6,B0000KC2BK,"Knorr Granulated Bouillon, Chicken 15.9 oz",NaN,$3.09,NaN
7,B0001E1IN8,"Castillo Habanero Hot Sauce Red, 5 oz",NaN,NaN,NaN
8,B00032E8XK,"Chicken of the Sea, Solid White Albacore Tuna ...","[B000XDIDOO, B000WLEL84]",$1.48,"[B00OKB7W6A, B00OKB7CRE, B0014D0UBG, B07BQZHBV..."
9,B0005XMTHE,"Smucker's Concord Grape Jam, 32 oz",NaN,$2.29,"[B014M6LTSO, B00I8G6D6I, B00PC5CI8K, B01KPSBV9U]"


In [28]:
#Remove duplicates asin
meta_df.drop_duplicates(subset ="asin", keep = 'first', inplace = True) 
rows,columns=meta_df.shape

In [61]:
df2 = meta_df.set_index("asin", drop = False)
unique_asin = df2['asin'].unique()

In [77]:
# Replace product asin into title
title = meta_df[['title']]
type(title)



pandas.core.frame.DataFrame

In [90]:
# Function to replace asin by title
def replace_with_title(col_index):
    ret = []
    for i in range(0, rows):
        t_list = []
        asin_list = meta_df.values[i,col_index]
        if type(asin_list) != float:
            for item in asin_list:
                if item in unique_asin:
                    tit = df2.loc[str(item), 'title']
                    t_list.append(tit)
                else:
                    t_list.append(item)
        ret.append(t_list)   
    return ret

# Replace title for also buy
ab_list = replace_with_title(2)

# Replace title for also view
av_list = replace_with_title(4)

# Problem, there are also buy and view products that belong to different categories

In [92]:
title['also_buy'] = ab_list
title['also_view'] = av_list

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [93]:
title

,title,also_buy,also_view
0,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[Fritos Original Corn Chips, 9.25 Ounce, B007N...",[]
2,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]","[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81..."
3,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...","[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9..."
4,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],[Glad ClingWrap Plastic Wrap - 200 Square Foot...
5,"Ibarra Mexican Chocolate, 19 oz",[B008MHKWLK],[]
6,"Knorr Granulated Bouillon, Chicken 15.9 oz",[],[]
7,"Castillo Habanero Hot Sauce Red, 5 oz",[],[]
8,"Chicken of the Sea, Solid White Albacore Tuna ...","[Joy Dishwashing Liquid, Lemon Scent, 30 Ounce...",[Chicken of the Sea White Albacore Tuna in Wat...
9,"Smucker's Concord Grape Jam, 32 oz",[],"[Jif Creamy Peanut Butter Twin Pack, 80 Ounce,..."
10,"Snapple Peach Tea, 32 fl oz bottle","[B0041LYS66, Sweet Baby Ray's Barbecue Sauce, ...","[B0041LYS66, B011ENMK1S]"


In [94]:
def together_product(df, col_index):
    ret = []
    for i in range(0, rows):
        item = [df.values[i, 0]]
        prod = df.values[i, col_index]
        if type(prod) != float:
            item += prod
        ret.append(item)
    return ret

In [95]:
# Create also buy table 
# type(asin) = unicode
also_buy = together_product(title, 1) #Or replace title with meta_df

In [96]:
# Create also buy table
also_view = together_product(title, 2)

In [100]:
len(also_view)

10814

In [101]:
len(also_buy)

10814

https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/

In [102]:
av_association_rules = apriori(also_view, min_support=0.005, min_confidence=0.2, min_lift=1, min_length=2)
av_association_results = list(av_association_rules)

In [103]:
print(len(av_association_results))
print(av_association_results[0])

1380
RelationRecord(items=frozenset([u"Annie Chun's Noodle Bowl, Korean Sweet Chili, 7.9 Ounce", u"Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce"]), support=0.005918254115036064, ordered_statistics=[OrderedStatistic(items_base=frozenset([u"Annie Chun's Noodle Bowl, Korean Sweet Chili, 7.9 Ounce"]), items_add=frozenset([u"Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce"]), confidence=0.7710843373493975, lift=77.92996284202229), OrderedStatistic(items_base=frozenset([u"Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce"]), items_add=frozenset([u"Annie Chun's Noodle Bowl, Korean Sweet Chili, 7.9 Ounce"]), confidence=0.5981308411214954, lift=77.9299628420223)])


In [108]:
ab_association_rules = apriori(also_buy, min_support=0.005, min_confidence=0.2, min_lift=1, min_length=2)
ab_association_results = list(ab_association_rules)

KeyboardInterrupt: 

In [106]:
def print_assoc(association_results):
    for item in association_results:

        # first index of the inner list
        # Contains base item and add item
        pair = item[0] 
        items = [x for x in pair]
        print("Rule: " + items[0] + " -> " + items[1])

        #second index of the inner list
        print("Support: " + str(item[1]))

        #third index of the list located at 0th
        #of the third index of the inner list

        print("Confidence: " + str(item[2][0][2]))
        print("Lift: " + str(item[2][0][3]))
        print("=====================================")

In [107]:
print_assoc(av_association_results)
#print_assoc(ab_association_results)

Rule: Annie Chun's Noodle Bowl, Korean Sweet Chili, 7.9 Ounce -> Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce
Support: 0.00591825411504
Confidence: 0.771084337349
Lift: 77.929962842
Rule: Annie Chun's Noodle Bowl, Korean Sweet Chili, 7.9 Ounce -> Simply Asia Sesame Teriyaki Noodle Bowl, 8.5 oz
Support: 0.00508599963011
Confidence: 0.66265060241
Lift: 58.2593789793
Rule: Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce -> Annie Chun's Noodle Bowl, Teriyaki, Non-GMO, Vegan, 7.8 Ounce
Support: 0.00508599963011
Confidence: 0.514018691589
Lift: 85.5168943206
Rule: B00OKEUVXI -> Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce
Support: 0.0052709450712
Confidence: 0.532710280374
Lift: 68.5801068091
Rule: Simply Asia Sesame Teriyaki Noodle Bowl, 8.5 oz -> Annie Chun's Noodle Bowl, Pad Thai, Non-GMO, Vegan, 8.4 Ounce
Support: 0.00554836323285
Confidence: 0.560747663551
Lift: 49.3002051516
Rule: B01DBKRWWY -> B00I8DS5DA
Support: 0.00601072683558
Confide

Lift: 42.7450706845
Rule: Barilla Pasta, Elbow, 16 Ounce -> Barilla Pasta, Spaghetti, 16 Ounce
Support: 0.00573330867394
Confidence: 0.553571428571
Lift: 51.6062192118
Rule: Barilla Pasta, Elbow, 16 Ounce -> Barilla Pasta, Veggie Rotini, 12 Ounce
Support: 0.0052709450712
Confidence: 0.508928571429
Lift: 57.3286830357
Rule: Barilla Pasta, Elbow, 16 Ounce -> Jell-O Raspberry Gelatin Mix 3 Ounce Box
Support: 0.0052709450712
Confidence: 0.508928571429
Lift: 31.0935230024
Rule: Barilla Pasta, Farfalle, 16 Ounce -> Barilla Pasta, Rotini, 16 Ounce
Support: 0.00943221749584
Confidence: 0.744525547445
Lift: 41.933850365
Rule: Barilla Pasta, Farfalle, 16 Ounce -> Barilla Pasta, Spaghetti, 16 Ounce
Support: 0.00675050859996
Confidence: 0.532846715328
Lift: 49.6741756859
Rule: Barilla Pasta, Farfalle, 16 Ounce -> Barilla Pasta, Veggie Rotini, 12 Ounce
Support: 0.00582578139449
Confidence: 0.459854014599
Lift: 51.8006386861
Rule: Barilla Pasta, Farfalle, 16 Ounce -> Jell-O Raspberry Gelatin Mix 3 O

Support: 0.00536341779175
Confidence: 0.41726618705
Lift: 53.7180541281
Rule: Glade Solid Air Freshener, Clean Linen, 6 oz -> B00RBPZEG2
Support: 0.00591825411504
Confidence: 0.460431654676
Lift: 60.7208282155
Rule: Jell-O Raspberry Gelatin Mix 3 Ounce Box -> B00RBPZEG2
Support: 0.00628814499723
Confidence: 0.489208633094
Lift: 57.5032843291
Rule: Mr. Clean with Febreze Meadows and Rain Multi-Surface Cleaner, 40 fl oz -> B00RBPZEG2
Support: 0.00508599963011
Confidence: 0.395683453237
Lift: 55.5704008222
Rule: Glade Solid Air Freshener, Clean Linen, 6 oz -> Jell-O Raspberry Gelatin Mix 3 Ounce Box
Support: 0.00638061771777
Confidence: 0.496402877698
Lift: 53.6810071942
Rule: Maxwell House International Cafe Flavored Instant Coffee, Cafe Francais, 7.6 Ounce Canister -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00591825411504
Confidence: 0.460431654676
Lift: 58.5777401608
Rule: Glade Solid Air Freshener, Clean Linen, 6 oz -> Mr. Clean with Febreze Meadows and Rain Multi-Surf

Rule: B06X9WNQFV -> B01LWDYLAH
Support: 0.00712039948215
Confidence: 0.353211009174
Lift: 35.0424206717
Rule: B01LWDYLAH -> B076CQ1H96
Support: 0.00656556315887
Confidence: 0.325688073394
Lift: 40.4826531688
Rule: Cheez-It Baked Snack Cheese Crackers, White Cheddar, 7 oz Box -> B01LWDYLAH
Support: 0.00564083595339
Confidence: 0.279816513761
Lift: 43.2276539974
Rule: B01LWDYLAH -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00665803587942
Confidence: 0.330275229358
Lift: 41.5301898869
Rule: B01LWDYLAH -> Jell-O Raspberry Gelatin Mix 3 Ounce Box
Support: 0.00841501756982
Confidence: 0.417431192661
Lift: 40.6675758327
Rule: Maxwell House International Cafe Flavored Instant Coffee, Cafe Francais, 7.6 Ounce Canister -> B01LWDYLAH
Support: 0.0054558905123
Confidence: 0.270642201835
Lift: 45.026534933
Rule: B01LWDYLAH -> Mr. Clean with Febreze Meadows and Rain Multi-Surface Cleaner, 40 fl oz
Support: 0.00647309043832
Confidence: 0.321100917431
Lift: 41.3379204893
Rule: B07C4BMHH3 

Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00601072683558
Confidence: 0.467625899281
Lift: 58.1253617795
Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00582578139449
Confidence: 0.453237410072
Lift: 61.2663669065
Rule: B01L1GEY8U -> Cheez-It Baked Snack Cheese Crackers, White Cheddar, 7 oz Box
Support: 0.00508599963011
Confidence: 0.395683453237
Lift: 54.1635552318
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00656556315887
Confidence: 0.510791366906
Lift: 56.9453385745
Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00702792676161
Confidence: 0.546762589928
Lift: 52.3246959954
Rule: B01L1GEY8U -> Maxwell House International Cafe Flavored Instant Coffee, Cafe Francais, 7.6 Ounce Canister
Support: 0.00536341779175
Confidence: 0.41726618705
Lift: 59.3725861416
Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00582578139449
Confidence: 0.453237410072
Lift: 56.3368891094
Rule: B01L1GEY8U -> Jell-O Raspberry Gelatin Mix 3 Ounce Box
Support: 0.00582578139449
Confidence: 0.453237

Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00554836323285
Confidence: 0.431654676259
Lift: 64.8321342926
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00591825411504
Confidence: 0.460431654676
Lift: 58.5777401608
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00508599963011
Confidence: 0.395683453237
Lift: 64.8321342926
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00554836323285
Confidence: 0.431654676259
Lift: 61.4199166982
Rule: B01L1GEY8U -> Maxwell House International Cafe Flavored Instant Coffee, Cafe Francais, 7.6 Ounce Canister
Support: 0.00508599963011
Confidence: 0.395683453237
Lift: 61.1274409044
Rule: B01L1GEY8U -> B01LWDYLAH
Support: 0.00536341779175
Confidence: 0.41726618705
Lift: 57.850212138
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Linen, 6 oz
Support: 0.00508599963011
Confidence: 0.395683453237
Lift: 67.9193787827
Rule: B01L1GEY8U -> Glade Solid Air Freshener, Clean Line

In [ ]:
print_assoc(ab_association_results)